In [ ]:
def vlmate_pipeline(image_path, record_seconds=5):
    print("Recording your question...")
    record(record_seconds)
    print("Converting speech to text...")
    result = model.transcribe("recorded.wav")
    question = result["text"]
    print("You asked:", question)
    image = Image.open(image_path)

    messages = [
        {"role": "user", "content": [
            {"type": "image"},
            {"type": "text", "text": question}
        ]}
    ]
    input_text = tokenizer.apply_chat_template(messages, add_generation_prompt=True)
    inputs = tokenizer(
        image,
        input_text,
        add_special_tokens=False,
        return_tensors="pt"
    ).to("cuda")

    from transformers import TextStreamer
    text_streamer = TextStreamer(tokenizer, skip_prompt=True)
    print("Thinking...")
    response_ids = model_vlm.generate(**inputs, max_new_tokens=128, use_cache=True)
    response = tokenizer.decode(response_ids[0], skip_special_tokens=True)
    print("VLM Response:", response)

    tts = gTTS(text=response, lang='en')
    tts.save("response.mp3")

    print("Speaking the answer...")
    return Audio("response.mp3", autoplay=True)

In [ ]:
vlmate_pipeline('/content/IMG_6812.jpeg', record_seconds=5)